# Ensebles

Un ensemble è una collezione di classificatori, tutti addestrati per performare lo stesso compito.

Un ensemble può consistere in diverse versioni dello stesso modello, o di modelli diversi.

L'output finale per un classificatore ensemble è tipicamente ottenere attraverso una media (pesata) delle predictions dei singoli modelli dell'ensemble.

Un ensemble di diversi modelli che conseguono una simile performance generalizzativa, spesso superano in performance i risultati dei singoli modelli.

---

# Ensemble intuition

Supponiamo di avere un ensemble di funzioni di classificatori binari $f_k(x)$ per $k = 1,...,K$.

Supponiamo inoltre che mediamente abbiano lo stesso errore di classificazione $\epsilon = E_{p(x,y)}[y \neq f_k(x)] < 0.5$, ma che gli errori che commettono siano indipendenti.

L'intuizione sta nel fatto che la maggioranza dei $K$ classificatori, dovrebbe classificare correttamente un dato esempio, dove un singolo classificatore potrebbe fallire.

Un semplice voto di maggioranza potrebbe migliorare la performance di classificazione diminuendo la varianza.

---

# Proof of Majority Voting

Supponiamo di avere un ensemble di $K$ classificatori binari, dove ogni classificatore ha una accuracy $\alpha \gt 0.5$.

La maggioranza dei votanti commette un errore quando $\frac{K}{2} + 1$ classificatori sbagliano.

L'errore segue la distribuzione binomiale:

#### $P(x \leq k) = \sum_{i=0}^{k} \binom{K}{k} \alpha^i (1-\alpha)^{K-i}$

---

## Esempio

Si suppone di avere 5 classificatori con accuracy $\alpha = 0.75$. L'errore di probabilità dell'ensemble è: (l'ensemble sbaglia quando almeno 3 classificatori sbaglianom da questo $x \leq 2$)

### $P(x \leq 2) = \sum_{i=0}^{2} \binom{5}{i} 0.75^i (0.25)^{5-i} = 0.105$

L'accuratezza dell'ensemble è $1-P(x \leq 2) = 0.895$.

<span style="color:gold;">Questo è vero quando gli errori tra i classificatori non sono correlati</span>

---

# Independent Training Sets

Supponiamo di collezionare diversi training set indipendenti $T_{r1}, ..., T_{rK}$ e di usarli per addestrare diverse istanze dello stesso classificatore ottenendo $K$ funzioni di classificazione $f_1(x), ..., f_K(x)$.

I classificatori addestrati in questo modo sono garantiti di commettere errori indipendenti sui dati di test.

Se l'errore atteso di ogni classificatore è minore di 0.5, allora il voto di maggioranza pesato è garantito di ridurre l'errore di generalizzazione atteso.

<span style="color:gold;">Qual è il punto debole di questo approccio?</span>